<h1 style="color:orange; font-size:48px; text-align:center">Topic 3: Extract insights from text with the Azure AI Language service</h1>
 
The Language service is an Azure AI service that supports analysis of text, including language detection, sentiment analysis, key phrase extraction, and entity recognition.

For example, suppose a travel agency wants to process hotel reviews that have been submitted to the company’s web site. By using the Language service, they can determine the language each review is written in, the sentiment (positive, neutral, or negative) of the reviews, key phrases that might indicate the main topics discussed in the review, and named entities, such as places, landmarks, or people mentioned in the reviews.

Install the dependencies for text analytics. This contains the SDK to call the azure text-analytics API.

In [1]:
!pip install azure-ai-==5.3.0

Note: you may need to restart the kernel to use updated packages.


Now that you have created a client for the Text Analytics API, we can use it to detect the language, Evaluate sentiment, Identify key phrases, Extract entities and Extract linked entities from the text. Read the comments in the code below to understand how to use the client to call the API.

## How to get KEY & ENDPOINT?

To create an Azure AI Services multi-service account and obtain the endpoint and key for a Python client app, you can follow these steps:

Step 1: Sign in to Azure Portal
- If you don't have an Azure account, you'll need to sign up for one. Once you have an account, log in to the Azure Portal.

Step 2: Create a New Azure AI Service Account

- In the Azure Portal, click on "Create a resource" in the left-hand menu.

- Search for "Azure AI" in the search bar and select "Azure AI."

- Click on "Create" to start configuring your Azure AI service account.

- In the "Basics" tab, provide the following information:

    - Subscription: Choose your Azure subscription.
    - Resource Group: Create a new resource group or select an existing one.
    - Region: Choose the region where you want to deploy the AI service.
    - Under the "Configuration" section, select "Multi-service."

- Provide a unique name for your multi-service account.

- Leave the rest of the options as their default values or configure them according to your needs.

- Review the configuration, and then click "Review + create."

- Review the summary, and if everything looks correct, click "Create" to create the multi-service account.

Step 3: Obtain Endpoint and Key
- After the multi-service account is created, follow these steps to obtain the endpoint and key:

- Once the deployment is complete, navigate to the multi-service account you just created in the Azure Portal.

- In the left-hand menu, under "Settings," click on "Keys and Endpoint."

- You will see two keys, usually named "Key 1" and "Key 2." Either key can be used for authentication. Copy one of the keys and store it securely.

- Below the keys, you will find the "Endpoint" URL. Copy the URL; this is the endpoint you will use to access the AI services.

In [8]:
# Reading Credentials from Excel
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

df_cred = pd.read_excel("credentials.xlsx")

endpoint = df_cred["endpoint"][0]
key = df_cred["key"][0]

# Create client using endpoint and key
credential = AzureKeyCredential(key)


cog_client = TextAnalyticsClient(endpoint=endpoint, credential=credential)


In [ ]:
cog_client.detect_language()

In [18]:
document = ["I had an absolutely amazing stay at Golden Peak Guest House! From the moment I arrived, I was impressed by the warm and friendly staff who went above and beyond to make me feel welcome. The rooms were not only impeccably clean and beautifully decorated, but they also offered breathtaking views of the surrounding mountains.",
           "یہ کلیدیں آپ کی Azure AI سروس API تک رسائی کے لیے استعمال ہوتی ہیں۔ اپنی چابیاں شیئر نہ کریں۔ انہیں محفوظ طریقے سے ذخیرہ کریں- مثال کے طور پر، Azure Key Vault کا استعمال کرتے ہوئے۔ ہم ان کلیدوں کو باقاعدگی سے دوبارہ تخلیق کرنے کی بھی تجویز کرتے ہیں۔ API کال کرنے کے لیے صرف ایک کلید ضروری ہے۔ پہلی کلید کو دوبارہ تخلیق کرتے وقت، آپ سروس تک مسلسل رسائی کے لیے دوسری کلید استعمال کر سکتے ہیں۔"]

In [19]:
res = cog_client.detect_language(documents=document)
len(res)

2

In [21]:
res[1]

DetectLanguageResult(id=1, primary_language=DetectedLanguage(name=Urdu, iso6391_name=ur, confidence_score=0.99), warnings=[], statistics=None, is_error=False, kind=LanguageDetection)

In [27]:

sentimentAnalysis = cog_client.analyze_sentiment(documents=document)
sentimentAnalysis[1]["sentiment"]

'neutral'

In [29]:
res = cog_client.extract_key_phrases(documents=document)

In [32]:
res[0]["key_phrases"]

['Golden Peak Guest House',
 'amazing stay',
 'friendly staff',
 'breathtaking views',
 'surrounding mountains',
 'moment',
 'warm',
 'rooms']

In [34]:
I had an absolutely amazing stay at 
Golden Peak Guest House! From the moment I arrived, 
I was impressed by the warm and friendly staff who 
went above and beyond to make me feel welcome.
The rooms were not only impeccably clean and beautifully decorated, 
but they also offered breathtaking views of the surrounding mountains.

SyntaxError: EOL while scanning string literal (1348922731.py, line 3)

In [35]:
entities = cog_client.recognize_entities(documents=document)
entities[0]

RecognizeEntitiesResult(id=0, entities=[CategorizedEntity(text=Golden Peak Guest House, category=Location, subcategory=None, length=23, offset=36, confidence_score=0.89), CategorizedEntity(text=staff, category=PersonType, subcategory=None, length=5, offset=129, confidence_score=0.83), CategorizedEntity(text=rooms, category=Location, subcategory=None, length=5, offset=190, confidence_score=0.71), CategorizedEntity(text=mountains, category=Location, subcategory=Geographical, length=9, offset=314, confidence_score=0.91)], warnings=[], statistics=None, is_error=False, kind=EntityRecognition)

In [10]:


# Get language
detectedLanguage = cog_client.detect_language(documents=document)[0]
print('\nLanguage: {}'.format(detectedLanguage.primary_language.name))

# Get sentiment
sentimentAnalysis = cog_client.analyze_sentiment(documents=document)[0]
print("\nSentiment: {}".format(sentimentAnalysis.sentiment))


# Get key phrases
phrases = cog_client.extract_key_phrases(documents=document)[0].key_phrases
if len(phrases) > 0:
    print("\nKey Phrases:")
    for phrase in phrases:
        print('\t{}'.format(phrase))

# Get entities
entities = cog_client.recognize_entities(documents=document)[0].entities
if len(entities) > 0:
    print("\nEntities")
    for entity in entities:
        print('\t{} ({})'.format(entity.text, entity.category))


# Get linked entities
entities = cog_client.recognize_linked_entities(documents=document)[0].entities
if len(entities) > 0:
    print("\nLinks")
    for linked_entity in entities:
        print('\t{} ({})'.format(linked_entity.name, linked_entity.url))



Language: English

Sentiment: positive

Key Phrases:
	Golden Peak Guest House
	amazing stay
	friendly staff
	breathtaking views
	surrounding mountains
	moment
	warm
	rooms

Entities
	Golden Peak Guest House (Location)
	staff (PersonType)
	rooms (Location)
	mountains (Location)

Links
	Spantik (https://en.wikipedia.org/wiki/Spantik)
	Guest house (https://en.wikipedia.org/wiki/Guest_house)


In [4]:
# Analyze each text file in the reviews folder
import os
reviews_folder = 'data/reviews'
for file_name in os.listdir(reviews_folder):
    # Read the file contents
    print('\n-------------\n' + file_name)
    text = open(os.path.join(reviews_folder, file_name), encoding='utf8').read()
    print('\n' + text)

    # Get language
    detectedLanguage = cog_client.detect_language(documents=[text])[0]
    print('\nLanguage: {}'.format(detectedLanguage.primary_language.name))


    # Get sentiment
    sentimentAnalysis = cog_client.analyze_sentiment(documents=[text])[0]
    print("\nSentiment: {}".format(sentimentAnalysis.sentiment))


    # Get key phrases
    phrases = cog_client.extract_key_phrases(documents=[text])[0].key_phrases
    if len(phrases) > 0:
        print("\nKey Phrases:")
        for phrase in phrases:
            print('\t{}'.format(phrase))

    # Get entities
    entities = cog_client.recognize_entities(documents=[text])[0].entities
    if len(entities) > 0:
        print("\nEntities")
        for entity in entities:
            print('\t{} ({})'.format(entity.text, entity.category))


    # Get linked entities
    entities = cog_client.recognize_linked_entities(documents=[text])[0].entities
    if len(entities) > 0:
        print("\nLinks")
        for linked_entity in entities:
            print('\t{} ({})'.format(linked_entity.name, linked_entity.url))



-------------
review1.txt

Good Hotel and staff
The Royal Hotel, London, UK
3/2/2018
Clean rooms, good service, great location near Buckingham Palace and Westminster Abbey, and so on. We thoroughly enjoyed our stay. The courtyard is very peaceful and we went to a restaurant which is part of the same group and is Indian ( West coast so plenty of fish) with a Michelin Star. We had the taster menu which was fabulous. The rooms were very well appointed with a kitchen, lounge, bedroom and enormous bathroom. Thoroughly recommended.

Language: English

Sentiment: positive

Key Phrases:
	The Royal Hotel
	Good Hotel
	good service
	great location
	Buckingham Palace
	Westminster Abbey
	same group
	West coast
	Michelin Star
	taster menu
	enormous bathroom
	Clean rooms
	staff
	London
	UK
	stay
	courtyard
	restaurant
	part
	plenty
	fish
	kitchen
	lounge
	bedroom

Entities
	Hotel (Location)
	staff (PersonType)
	The Royal Hotel (Location)
	London (Location)
	UK (Location)
	3/2/2018 (DateTime)
	Buckin

# Translate Text
The Translator service is an Azure AI service that enables you to translate text between languages.

For example, suppose a travel agency wants to examine hotel reviews that have been submitted to the company’s web site, standardizing on English as the language that is used for analysis. By using the Translator service, they can determine the language each review is written in, and if it is not already English, translate it from whatever source language it was written in into English.

The Translator service can automatically detect the source language of text to be translated, but it also enables you to explicitly detect the language in which text is written.

In [42]:
import os
import requests, json

# Get Configuration Settings
cog_key = "8a0d5d61676d4f1dadba1a87a7a0c8fe"
cog_region = "eastus2"
translator_endpoint = 'https://api.cognitive.microsofttranslator.com/'

In [43]:
# Analyze each text file in the reviews folder
text = "Il s'agit d'un vieil hôtel (qui existe depuis les années 1950) et le mobilier des chambres est moyen - il devient un peu vieux maintenant et doit être changé. Internet ne fonctionnait pas et j'ai dû me rendre dans l'un de leurs bureaux pour m'enregistrer pour mon vol de retour. Le site Internet indique que c'est proche du British Museum, mais c'est trop loin pour y aller à pied."

source_language = cog_client.detect_language(documents=[text])[0]["primary_language"]["iso6391_name"]

In [47]:
# Use the Translator translate function
path = '/translate'
url = translator_endpoint + path

# Build the request
params = {
    'api-version': '3.0',
    'from': source_language,
    'to': ['en', "ur"]
}

headers = {
    'Ocp-Apim-Subscription-Key': cog_key,
    'Ocp-Apim-Subscription-Region': cog_region,
    'Content-type': 'application/json'
}

body = [{
    'text': text
}]

# Send the request and get response
result = requests.post(url, params=params, headers=headers, json=body)
response = result.json()
response

[{'translations': [{'text': "This is an old hotel (which has been around since the 1950s) and the furniture in the rooms is average - it's getting a bit old now and needs to be changed. The internet wasn't working and I had to go to one of their offices to check in for my flight home. The website says it's close to the British Museum, but it's too far to walk there.",
    'to': 'en'},
   {'text': 'یہ ایک پرانا ہوٹل ہے (جو 1950 کی دہائی سے موجود ہے) اور کمروں میں فرنیچر اوسط ہے - یہ اب تھوڑا پرانا ہو رہا ہے اور اسے تبدیل کرنے کی ضرورت ہے۔ انٹرنیٹ کام نہیں کر رہا تھا اور مجھے اپنے گھر جانے کے لئے چیک ان کرنے کے لئے ان کے ایک دفتر جانا پڑا۔ ویب سائٹ کا کہنا ہے کہ یہ برٹش میوزیم کے قریب ہے، لیکن وہاں چلنا بہت دور ہے۔',
    'to': 'ur'}]}]


<h1 style="color:red; font-size:48px; text-align:center">Home Task: Language Translation with Azure AI</h1>

In this home task, you will practice using Azure AI's Language Translation service to translate text from various languages into English. You will work with at least 10 text records or reviews written in a minimum of 5 different languages. Follow the instructions below to complete the task:

## Task Instructions:

### Collect Text Data

Gather at least 10 text records or reviews written in different languages. You can use online sources, books, articles, or any text in languages other than English.

### Translate Text

Write a Python script that uses the Azure AI Language Translation service to translate each text record into English. You can use the following steps as a guide:

- a. Import the required libraries.

- b. Define your Azure AI service credentials, including the API key, region, and endpoint. Use the provided code snippet as an example to set up your configuration.

- c. For each text record, use the Translator service to detect the language it is written in. You can follow the example code provided in your lab content under the "source_language" variable.

- d. If the detected language is not already English, use the Translator service to translate the text into English. Refer to the example code provided under the "Translation" section in your lab content.

- e. Save the original text and the translated text along with the detected language into a data structure for later analysis.

### Run the Translation Script

Execute your Python script to perform the translations for all 10 text records.

### Record the Results

Record the original text, translated text, and detected language for each text record in a table or spreadsheet.

In [ ]:
cog_client.